In [37]:
# imports
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium. webdriver. chrome. service import Service
from selenium. webdriver. chrome. options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from tqdm import tqdm
import pickle
from time import sleep

In [40]:
# district_urlsから特定の都道府県のurlを抽出
with open('district_urls.pkl', 'rb') as f:
    district_urls = pickle.load(f)

# urlを抽出
district_url = district_urls[42]
district_url

'https://www.beauty-park.jp/kumamoto/cities'

In [41]:
# 出力ファイル
OUTPUT_FILE = "shop_url_list_by_district/shop_url_list_kumamoto.pkl"

In [42]:
# city_urlsを取得
res = requests.get(district_url)
soup = BeautifulSoup(res.text, "html.parser")
selection_tab =soup.find(class_="selection-tab")
BASE_URL = "https://www.beauty-park.jp"
city_urls = [f'{BASE_URL}{a['href']}' for a in selection_tab.find_all('a', href=True)]

In [43]:
options = Options ()
options.binary_location = "/usr/bin/chromium-browser" # Chromium のパス
# options.add_argument ("--headless")
options.add_argument ("--no-sandbox")
options.add_argument ("--disable-dev-shm-usage")
service = Service ("/usr/bin/chromedriver") # chromedriverのパス

driver = webdriver.Chrome(service=service, options=options)


In [44]:
# shop_url_list_by_districtを取得
shop_url_list_by_district = []
for city_url in tqdm(city_urls):
    driver.get(city_url)
    sleep(1)
    shop_urls_by_city = []
    while True:
        # h4要素が読み込まれるまで待機
        wait = WebDriverWait(driver, 10)
        cards = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "h4")))
        shop_urls = []
        for card in cards:
            shop_url = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            shop_urls.append(shop_url)
        shop_urls_by_city.extend(shop_urls)
        try:
            next_button = driver.find_element(By.LINK_TEXT, "次を表示")
            next_button.click()
            sleep(2)
        except:
            break    
    shop_url_list_by_district.extend(shop_urls_by_city)
driver.quit()

100%|██████████| 24/24 [01:25<00:00,  3.56s/it]


In [45]:
print(len(shop_url_list_by_district))
print(len(set(shop_url_list_by_district)))
shop_url_list_by_district = list(set(shop_url_list_by_district))
print(len(shop_url_list_by_district))
print(len(set(shop_url_list_by_district)))

757
205
205
205


In [46]:
# 出力
with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump(shop_url_list_by_district, f)